# Exposed tweet collector (original tweet & reply containing politifact article)

In [1]:
# ******* Insert Twitter API Bearer Token here. Academic API required. *******
bearer_token = "AAAAAAAAAAAAAAAAAAAAAM%2BxZwEAAAAADgU3Yv5FyWJOF8q5Z8xMJwZydiE%3DBlIkDPQSVgxL7pwLKbHHyji7wTvTseFX8wgiPzvCmZ2FHUy7os"

## Dependency

In [2]:
cd /content/drive/MyDrive/

[Errno 2] No such file or directory: '/content/drive/MyDrive/'
/Users/JisangYu/Documents/UChicago_MPCS/Spring 2022/CMSC35401/hcml_factcheck


In [3]:
import requests
from tqdm.auto import tqdm
import time
import pickle
from glob import glob
import gc
import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
tqdm.pandas()

In [4]:
!git clone https://github.com/oduwsdl/tweetedat

fatal: destination path 'tweetedat' already exists and is not an empty directory.


In [5]:
# To identify timestamp from tweet id

def find_tweet_timestamp_post_snowflake(tid):
    offset = 1288834974657
    tstamp = (tid >> 22) + offset
    return tstamp

def find_tweet_timestamp_pre_snowflake(tid):
    with open(os.path.join('./tweetedat/data', "TweetTimeline.txt"), "r") as file_tweet_timeline:
        prev_line_parts = file_tweet_timeline.readline().rstrip().split(",")
        if tid < int(prev_line_parts[0]):
            return -1
        elif tid == int(prev_line_parts[0]):
            return int(prev_line_parts[1]) * 1000
        else:
            for line in file_tweet_timeline:
                line_parts = line.rstrip().split(",")
                if tid == int(line_parts[0]):
                    return int(prev_line_parts[1]) * 1000
                if int(prev_line_parts[0]) < tid < int(line_parts[0]):
                    estimated_timestamp = round(int(prev_line_parts[1]) + (((tid - int(prev_line_parts[0])) / (int(line_parts[0]) - int(prev_line_parts[0]))) * (int(line_parts[1]) - int(prev_line_parts[1]))))
                    return estimated_timestamp * 1000
                else:
                    prev_line_parts = line_parts
    return -1

def find_tweet_timestamp(tid):
    pre_snowflake_last_tweet_id = 29700859247
    if tid < pre_snowflake_last_tweet_id:
        tweet_timestamp = find_tweet_timestamp_pre_snowflake(tid)
    else:
        tweet_timestamp = find_tweet_timestamp_post_snowflake(tid)
    return tweet_timestamp


## Collect every tweet that contains a link to PolitiFact articles & replies to somebody

In [6]:
results = []
user_dict = {}
start_date = '2020-03-30T00:00:23.000Z' # first date of the data
last_date = '2021-03-30T00:00:23.000Z' # last date of the data (Recent data are collected earlier)

while True:
    url = 'https://api.twitter.com/2/tweets/search/all'

    headers = {'Authorization': bearer_token}

    querystring = {"query":'url:politifact.com is:reply', 
                    "end_time": last_date,
                    'max_results': 500, 
                    'tweet.fields': 'id,text,created_at,author_id,conversation_id,referenced_tweets,entities,public_metrics,lang',
                    'expansions': 'author_id,in_reply_to_user_id'}
    pbar = tqdm()

    while True:
        while True:
            response = requests.request("GET", url, headers=headers, params=querystring) # request tweets
            response_json = response.json()
            try:
                results += response_json['data']
                break
            except KeyError:
                time.sleep(60 * 11) # If facing the error due to the rate limit, rest 11 mins

        for user in response_json['includes']['users']:
            user_dict[user['id']] = user

        # When reaching the last page for the current end_time, break to update end_time to last_date
        if 'next_token' not in response.json()['meta']: 
            break
        last_date = response_json['data'][len(response_json['data'])-1]['created_at']
        next_token = response_json['meta']['next_token']
        querystring['next_token'] = next_token
        pbar.update(len(response_json['data']))
        time.sleep(1)

    pickle.dump(results, open('./data_temp/results_'+str(last_date[:10])+'_'+str(pbar.n)+'.pkl', 'wb')) # tweets are saved here
    # pickle.dump(user_dict, open('./data/user_dict_final.pkl', 'wb')) # save dictionary (user id -> user screenname)
    results = []

    if last_date < start_date:
        break

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [7]:
response_json

{'title': 'Unauthorized',
 'type': 'about:blank',
 'status': 401,
 'detail': 'Unauthorized'}

## Combine data files, data cleaning

In [ ]:
filelist = glob('./data_temp/results_*.pkl')
results = []
# combine data files
for f in tqdm(filelist):
    results += pickle.load(open(f, 'rb'))

In [ ]:
# data cleaning
already = set()
reply_results = []
for i in tqdm(range(len(results))):
    # check if the tweet is reply
    if 'in_reply_to_user_id' not in results[i] or 'referenced_tweets' not in results[i]:
        continue

    # include only english tweets, same author excluded
    if results[i]['lang'] != 'en' or results[i]['in_reply_to_user_id'] == results[i]['author_id']: 
        continue

    # check if politifact fact-checking "article" is included in the tweet
    contain_link = False
    if 'entities' in results[i]:
            if 'urls' in results[i]['entities']:
                for j in results[i]['entities']['urls']:
                    if 'politifact.com/factchecks' in j['expanded_url']: 
                        contain_link = True
    if not contain_link:
        continue

    # drop duplicates
    for j in range(len(results[i]['referenced_tweets'])):
        if results[i]['referenced_tweets'][j]['type'] == 'replied_to' and results[i]['id'] not in already:
            reply_results.append(results[i]) 
            already.add(results[i]['id'])

In [ ]:
# generate dateframe

pair_data = []
for i in tqdm(reply_results):
    original_tweet_id = i['referenced_tweets'][0]['id']
    original_user = i['in_reply_to_user_id']
    original_timestamp = find_tweet_timestamp(int(original_tweet_id)) / 1000
    original_at = datetime.datetime.strftime(datetime.datetime.fromtimestamp(original_timestamp), "%Y-%m-%dT%H:%M:%S.%fZ")

    reply_tweet_id = i['id']
    reply_user = i['author_id']
    reply_timestamp = datetime.datetime.strptime(i['created_at'], "%Y-%m-%dT%H:%M:%S.%fZ").timestamp()
    reply_at = i['created_at']
    reply_url = i['entities']['urls'][0]['expanded_url']

    reply_text = i['text']
    pair_data.append([original_tweet_id, original_user, original_timestamp, original_at, reply_tweet_id, reply_user, reply_timestamp, reply_at, reply_text, reply_url])

In [ ]:
df = pd.DataFrame(pair_data, columns=['original_tweet_id',
                                   'original_user', 
                                   'original_timestamp', 
                                   'original_at', 
                                   'reply_tweet_id', 
                                   'reply_user', 
                                   'reply_timestamp', 
                                   'reply_at', 
                                   'reply_text',
                                   'reply_url'])

## Collect user data & check if users are suspended

In [ ]:
users = sorted(list(set(list(df['original_user']) + list(df['reply_user']))))

In [ ]:
len(users)

In [ ]:
# get user data for both original tweet poster and reply poster from twitter api
# if users are suspended, data are not returned from api
users_data = []
for batch in tqdm(range(int(np.ceil(len(users) / 100)))):
    while True:
        querystring = {"ids":','.join(users[batch*100:(batch+1)*100]), 
                "user.fields": 'created_at,description,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}
        headers = {'Authorization': bearer_token}
        response = requests.request("GET", 'https://api.twitter.com/2/users', headers=headers, params=querystring)
        try:
            users_data += response.json()['data']
            break
        except KeyError:
            time.sleep(60)

In [ ]:
json.dump(users_data, open("./data/users_data.json", "w"))

In [ ]:
active_user_id = set()
for i in users_data:
    active_user_id.add(i['id'])

df['original_user_active'] = df['original_user'].progress_apply(lambda x: x in active_user_id)
df['reply_user_active'] = df['reply_user'].progress_apply(lambda x: x in active_user_id)

## Collect text of original tweet

In [ ]:
url = 'https://api.twitter.com/2/tweets'
headers = {'Authorization': bearer_token}

original_tweets = []
for batch in tqdm(range(np.ceil(len(df['original_tweet_id']) / 100).astype(int))):
    ids = ','.join(df['original_tweet_id'][batch*100:(batch+1)*100])
    querystring = {"ids":ids, 
                   'tweet.fields': 'id,text,created_at,author_id,conversation_id,referenced_tweets,public_metrics,lang'}
    response = requests.request("GET", url, headers=headers, params=querystring)
    original_tweets += response.json()['data']

In [ ]:
json.dump(original_tweets, open("./data/original_tweets.json", "w"))

In [ ]:
original_tweets_dict = {}
for i in original_tweets:
    original_tweets_dict[i['id']] = i['text']

In [ ]:
df['original_text'] = df['original_tweet_id'].progress_apply(lambda x: original_tweets_dict[x] if x in original_tweets_dict else None)

In [ ]:
df.to_csv('./data/exposed_tweet.csv', index=False)